<a href="https://colab.research.google.com/github/shaaagri/iat481-nlp-proj/blob/main/LLama2_vanilla_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
In this preparation notebook, we will set up a suitable variant of Llama2 LLM to be able to do some raw queries. We then will try to integrate it into a langchain-based pipeline and test it by asking the model some simple questions (a "vanilla" chatbot, missing yet any customization). This will help us to get ready for our project's next iteration (in another notebok) in which we will take this pipeline one step further by implementing RAG (Retrieval Augmented Generation).

*Note*: This notebook is partially based on `Chatbot_LLama_2.ipynb` from Maryiam's tutorials and also heavily borrows from `Run LLama-2 on Google Colab` tutorial by Muhammad Moin ([link](https://github.com/MuhammadMoinFaisal/LargeLanguageModelsProjects/blob/main/Run%20Llama2%20Google%20Colab/Llama_2_updated.ipynb])).

# Workflow

1. Gathering the Dependencies
2. Downloading and Loading the Model
3. Testing the Model with Raw Prompting

# Gathering the Dependencies


`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.



## Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).

In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).


## Installing the Packages

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --upgrade
!pip install huggingface_hub
!pip install llama-cpp-python

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 170.2 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 2.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Running command Getting requirements to build wheel
  Getting requirements to build wheel ... done
  Running command pip subprocess to install backend dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 39.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 23.6 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-ebwuv0qy/normal/local/bin
    changing

## Importing the libraries


In [3]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# Downloading and Loading the Model

Now that we have the required packages and libraries installed and imported, we can proceed to downloading and saving a quantized version of Llama-2-13b locally.

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
models_root = '/content/drive/MyDrive/IAT481/481 Team Projs/NLP Project/Models/'

In [6]:
%cd $models_root

/content/drive/.shortcut-targets-by-id/13FaXzyfvSXh_PD6h92caiYAKqKbPsFTR/481 Team Projs/NLP Project/Models


In [2]:
#model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
#model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

model_name_or_path = "TheBloke/Llama-2-7B-Chat-GGUF"
model_basename = "llama-2-7b-chat.Q5_K_S.gguf"

In [7]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-7b-chat.Q5_K_S.gguf:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

The model should be available from now locally on our google drive. Let's try to load it into Colab's T4 GPU and see the stats.

In [10]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=33 # Change this value based on your model and your GPU VRAM pool.
    )

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_co

Everything seems to be okay and we are lucky that, thanks to quantization, we are able to use this LLM right from our Colab notebook, for free.

# Testing the Model with Raw Prompting

First, let's construct the default prompt template that contains a simple system prompt:

In [11]:
def eval_default_prompt_templ(prompt):
  prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

  USER: %s

  ASSISTANT:
  '''

  return prompt_template % prompt

Now, without further ado, let's query the model with something and see what it gives us:

In [12]:
prompt='Where Simon Fraser University is located?'

In [13]:
response=lcpp_llm(prompt=eval_default_prompt_templ(prompt), max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

print(response)


llama_print_timings:        load time =     465.35 ms
llama_print_timings:      sample time =      60.65 ms /    83 runs   (    0.73 ms per token,  1368.49 tokens per second)
llama_print_timings: prompt eval time =     465.21 ms /    49 tokens (    9.49 ms per token,   105.33 tokens per second)
llama_print_timings:        eval time =    1957.62 ms /    82 runs   (   23.87 ms per token,    41.89 tokens per second)
llama_print_timings:       total time =    2773.32 ms /   131 tokens


{'id': 'cmpl-da4502e3-c734-4032-9084-0bd9b14a2b86', 'object': 'text_completion', 'created': 1712728440, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q5_K_S.gguf', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.\n\n  USER: Where Simon Fraser University is located?\n\n  ASSISTANT:\n   Simon Fraser University (SFU) is located in British Columbia, Canada. The main campus of SFU is situated in Burnaby, which is a city located in the Lower Mainland region of British Columbia, approximately 25 kilometers southeast of downtown Vancouver. The university has additional campuses in Surrey and Vancouver, as well as several satellite centers throughout BC.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 49, 'completion_tokens': 82, 'total_tokens': 131}}
